In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
! unzip /content/gdrive/MyDrive/PGSS2022/L5--7-15-2022/converted_keras.zip -d /content/

Archive:  /content/gdrive/MyDrive/PGSS2022/L5--7-15-2022/converted_keras.zip
 extracting: /content/keras_model.h5  
 extracting: /content/labels.txt     


In [4]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the model
model = load_model('keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

## Data Preparation

In [6]:
# Replace this with the path to your image
image = Image.open('/content/gdrive/MyDrive/PGSS2022/L5--7-15-2022/MaskData/with_mask/106-with-mask.jpg')
#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array


## Model Inference

In [8]:
# run the inference
prediction = model.predict(data)
print(prediction)


[[5.009072e-07 9.999995e-01]]


In [19]:
predictedClass = np.argmax(prediction, axis=1)
predictedClass

array([1])

In [20]:
import pandas as pd
labels = pd.read_csv('labels.txt', header=None, delimiter=' ')
labels.columns= ['Prediction', 'Label']
labels

,Prediction,Label
0,0,NoMask
1,1,Mask


In [32]:
PredictionLabel = labels.loc[labels['Prediction'].isin(predictedClass)]['Label'].values[0]
PredictionLabel

'Mask'

## Put the stuff that works into a script and run it

In [33]:
! touch myInferenceScript.py

In [34]:
! python myInferenceScript.py

2022-07-15 20:34:14.169511: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Mask


In [35]:
! touch myInferenceFunction.py

In [37]:
! python myInferenceFunction.py

2022-07-15 20:39:11.685401: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Mask


In [38]:
! touch InferenceFunctionWithArguments.py

In [39]:
! python myInferenceFunction.py labels.txt keras_model.h5 /content/gdrive/MyDrive/PGSS2022/L5--7-15-2022/MaskData/with_mask/106-with-mask.jpg

2022-07-15 20:42:39.767071: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Mask


In [41]:
! python myInferenceFunction.py labels.txt keras_model.h5 /content/gdrive/MyDrive/PGSS2022/L5--7-15-2022/MaskData/without_mask/101.jpg

2022-07-15 20:43:19.339381: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Mask


## Test python functions in the notebook

In [1]:
# Run me with: ! python InferenceFunctionWithArguments.py labels.txt keras_model.h5 testImage.jpg

from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
import sys

def ModelInference_Keras(labelFile='labels.txt', modelFile='keras_model.h5', imageFile='testImage.jpg'):
  # Load labels.txt
  labels = pd.read_csv(labelFile, header=None, delimiter=' ')
  labels.columns= ['Prediction', 'Label']

  # Load the model
  model = load_model(modelFile)

  # Load the data and transform it:
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
  # Replace this with the path to your image
  image = Image.open(imageFile)
  #resize the image to a 224x224 with the same strategy as in TM2:
  #resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.ANTIALIAS)
  #turn the image into a numpy array
  image_array = np.asarray(image)
  # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
  # Load the image into the array
  data[0] = normalized_image_array

  # Run the inference / prediction on the loaded data
  prediction = model.predict(data)
  predictedClass = np.argmax(prediction, axis=1)

  PredictionLabel = labels.loc[labels['Prediction'].isin(predictedClass)]['Label'].values[0]
  
  return(PredictionLabel)

myPredictions = ModelInference_Keras('labels.txt', 'keras_model.h5', '15.jpg') #sys.argv[0], sys.argv[1], sys.argv[2])
print(myPredictions)



2022-07-15 17:04:02.861992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/var/folders/vp/j3tfbs8x34n30v4fnpskn4dc0000gn/T/ipykernel_52664/1562180717.py:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = ImageOps.fit(image, size, Image.ANTIALIAS)


1/1 [==============================] - 1s 1s/step
NoMask
